# Import the required libraries

In [ ]:
import os
import zipfile
import requests
import pandas as pd
from sqlalchemy import create_engine, text

# Extract the OMOP dataset from dagshub repository

In [ ]:
from dagshub.streaming import DagsHubFilesystem

fs = DagsHubFilesystem(".", repo_url="https://dagshub.com/DagsHub-Datasets/cmsdesynpuf-omop-dataset")

fs.listdir("s3://synpuf-omop")

In [ ]:
# 1. Unmount any previous filesystem
try:
    del fs
except:
    pass

# 2. Import DagsHub streaming
from dagshub.streaming import DagsHubFilesystem
import os

# 3. Mount the repository
fs = DagsHubFilesystem(
    ".", 
    repo_url="https://dagshub.com/DagsHub-Datasets/cmsdesynpuf-omop-dataset"
)

# 4. Folder to download
source_folder = "s3://synpuf-omop/cmsdesynpuf100k"
local_folder = "cmsdesynpuf100k"

# Create local directory if needed
os.makedirs(local_folder, exist_ok=True)

# 5. List files inside the folder
files = fs.listdir(source_folder)
print("Files found:", files)

# 6. Download each file
for filename in files:
    src_path = f"{source_folder}/{filename}"
    dest_path = os.path.join(local_folder, filename)

    print(f"Downloading {filename} ...")

    with fs.open(src_path, "rb") as fsrc, open(dest_path, "wb") as fdst:
        fdst.write(fsrc.read())

print("Download complete!")



# Transform the zipped files into csv files

In [ ]:
import gzip
import shutil
from pathlib import Path

# 👉 Set this to the folder where your downloaded files are stored
data_dir = Path("/Users/maggie/Documents/data_career/Portfolio_Projects/epilepsy_study/cmsdesynpuf100k")

# Create an output directory (optional)
output_dir = data_dir / "csv_extracted"
output_dir.mkdir(exist_ok=True)

# Loop through all .csv.gz files
for gz_file in data_dir.glob("*.csv.gz"):
    csv_filename = gz_file.stem  # removes .gz
    csv_path = output_dir / csv_filename

    print(f"Extracting: {gz_file.name} -> {csv_filename}")

    # Decompress the file
    with gzip.open(gz_file, 'rb') as f_in:
        with open(csv_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

print("✔️ All .csv.gz files successfully extracted!")


In [ ]:
for gz_file in data_dir.glob("*.csv.gz"):
    gz_file.unlink()
print("🗑️ Deleted all .csv.gz files.")


# Install and import the required libraries then connect to PostgreSQL database

In [ ]:
!pip install psycopg2-binary pandas sqlalchemy tqdm

In [ ]:
from sqlalchemy import create_engine

# UPDATE these details to match your credentials
pg_user = "your_username"
pg_password = "your_password"
pg_host = "your_server_name"
pg_port = "your_port_number"
pg_db = "your_database_name"

engine = create_engine(
    f"postgresql://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_db}"
)

engine.connect()
print("Connected to PostgreSQL")


# Load the data from the csv files to your database

In [ ]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# UPDATE to your extracted CSV directory
csv_dir = Path("your/file/path/name")

# Get all CSV files
csv_files = list(csv_dir.glob("*.csv"))

print(f"Found {len(csv_files)} CSV files to load.")

for csv_file in tqdm(csv_files):
    table_name = csv_file.stem.lower()   # Use filename (no extension) as table
    
    print(f"\nLoading {csv_file.name} → {table_name}")

    # Load CSV in chunks to avoid memory issues
    chunksize = 50_000

    try:
        for chunk in pd.read_csv(csv_file, chunksize=chunksize):
            chunk.to_sql(
                table_name,
                engine,
                if_exists="append",   # append data if table exists
                index=False
            )
        print(f"Loaded: {table_name}")

    except Exception as e:
        print(f"Error loading {table_name}: {e}")


In [ ]:
tables = engine.table_names()
print("Tables in database:")
for t in tables:
    print(" -", t)
